In [1]:
import pathlib

import numpy as np
import pandas as pd
import umap

In [2]:
# set paths
# input path
data_path = pathlib.Path(
    "../../1.scDINO_run/outputdir/apoptosis_timelapse/CLS_features/CLS_features_annotated.parquet"
).resolve(strict=True)
# output path
output_path = pathlib.Path(
    "../../1.scDINO_run/outputdir/apoptosis_timelapse/CLS_features/CLS_features_annotated_umap.csv"
).resolve()
# shiny output path
shiny_output_path = pathlib.Path(
    "../../5.scDINO_analysis/2.scDINO_processing/temporal_shiny_app/CLS_features_annotated_umap.csv"
).resolve()

In [3]:
# load in data
cls_df = pd.read_parquet(data_path)
cls_df.head()

,Metadata_image_path,Metadata_Well,Metadata_FOV,Metadata_Time,Metadata_ImageNumber,Metadata_Nuclei_Number_Object_Number,Metadata_compound,Metadata_dose,Metadata_control,channel488-1_cls_feature_0,...,channel_DNA_cls_feature_90,channel_DNA_cls_feature_91,channel_DNA_cls_feature_92,channel_DNA_cls_feature_93,channel_DNA_cls_feature_94,channel_DNA_cls_feature_95,channel_DNA_cls_feature_96,channel_DNA_cls_feature_97,channel_DNA_cls_feature_98,channel_DNA_cls_feature_99
0,../../data/processed_images/crops/C-02/image_n...,C-02,0001,0010,10,10,Staurosporine,0.0,negative,-0.013294,...,-0.026835,-0.025143,0.022814,0.044236,-0.048172,0.003977,0.005565,0.033877,0.082223,0.009103
1,../../data/processed_images/crops/C-02/image_n...,C-02,0001,0010,10,100,Staurosporine,0.0,negative,0.003657,...,0.024010,0.002705,-0.059467,0.032855,-0.057530,0.031927,0.017482,0.051654,0.024463,-0.034733
2,../../data/processed_images/crops/C-02/image_n...,C-02,0001,0010,10,101,Staurosporine,0.0,negative,0.013813,...,-0.007732,0.024938,0.027292,0.034904,-0.127702,-0.014732,0.033218,0.008977,0.031269,-0.031651
3,../../data/processed_images/crops/C-02/image_n...,C-02,0001,0010,10,102,Staurosporine,0.0,negative,0.026417,...,-0.041826,-0.028302,-0.034485,0.032456,-0.054537,0.021049,0.028299,-0.006374,0.101494,-0.018018
4,../../data/processed_images/crops/C-02/image_n...,C-02,0001,0010,10,103,Staurosporine,0.0,negative,-0.002667,...,-0.029698,-0.017408,-0.011866,0.045469,-0.077298,0.000747,0.076386,-0.007291,0.015712,-0.011359


In [4]:
# get the metadata
metadata_df = cls_df.columns[cls_df.columns.str.contains("Metadata")]
metadata_df = cls_df[metadata_df]
feature_df = cls_df.drop(metadata_df.columns, axis=1)
print(f"metadata_df shape: {metadata_df.shape}")
print(f"feature_df shape: {feature_df.shape}")

metadata_df shape: (243970, 9)
feature_df shape: (243970, 1536)


In [5]:
# define the UMAP model
umap_model = umap.UMAP(
    n_components=2, random_state=0, n_neighbors=30, min_dist=0.1, metric="euclidean"
)

# fit the UMAP model
umap_embedding = umap_model.fit_transform(feature_df)
umap_embedding_df = pd.DataFrame(umap_embedding, columns=["UMAP1", "UMAP2"])
# add the metadata back
umap_embedding_df = pd.concat([metadata_df, umap_embedding_df], axis=1)
umap_embedding_df.head()

/home/lippincm/miniconda3/envs/scDINO_env/lib/python3.8/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


,Metadata_image_path,Metadata_Well,Metadata_FOV,Metadata_Time,Metadata_ImageNumber,Metadata_Nuclei_Number_Object_Number,Metadata_compound,Metadata_dose,Metadata_control,UMAP1,UMAP2
0,../../data/processed_images/crops/C-02/image_n...,C-02,0001,0010,10,10,Staurosporine,0.0,negative,0.062785,1.788185
1,../../data/processed_images/crops/C-02/image_n...,C-02,0001,0010,10,100,Staurosporine,0.0,negative,0.491567,5.255784
2,../../data/processed_images/crops/C-02/image_n...,C-02,0001,0010,10,101,Staurosporine,0.0,negative,-0.978864,2.113676
3,../../data/processed_images/crops/C-02/image_n...,C-02,0001,0010,10,102,Staurosporine,0.0,negative,1.878953,3.504759
4,../../data/processed_images/crops/C-02/image_n...,C-02,0001,0010,10,103,Staurosporine,0.0,negative,0.536983,3.264089


In [6]:
# save the UMAP embeddings to parquet
umap_embedding_df.to_csv(output_path)

# save the UMAP embeddings to shiny app
umap_embedding_df.to_csv(shiny_output_path)

OSError: Cannot save file into a non-existent directory: '/home/lippincm/Documents/4TB/data/live_cell_timelapse_apoptosis/5.scDINO_analysis/5.scDINO_analysis/2.scDINO_processing/temporal_shiny_app'